In [4]:
from sympy.physics.secondquant import *
from sympy import symbols,I
import sympy as sp
from no import normal

In [5]:
ind_a = symbols('a')
a = B(ind_a)
ad = Bd(ind_a)

In [8]:
g3 = symbols('g3', real=True)
g4 = symbols('g4', real=True)
g5 = symbols('g5', real=True)
g6 = symbols('g6', real=True)
omega = symbols('omega', real=True)
kappa = symbols('kappa', real=True)
v = normal( g4*(a+ad)**4  )
h0 = omega * ad * a

In [9]:
v

3*g4 + 6*g4*AnnihilateBoson(a)**2 + g4*AnnihilateBoson(a)**4 + 12*g4*CreateBoson(a)*AnnihilateBoson(a) + 4*g4*CreateBoson(a)*AnnihilateBoson(a)**3 + 6*g4*CreateBoson(a)**2 + 6*g4*CreateBoson(a)**2*AnnihilateBoson(a)**2 + 4*g4*CreateBoson(a)**3*AnnihilateBoson(a) + g4*CreateBoson(a)**4

In [10]:
ad

CreateBoson(a)

In [11]:
def remove_higher(expr):
    terms_higher = [
        term for term in expr.args if (term.has(g4**2) or term.has(g3**2*g4) or term.has(g3**4) or term.has(g3*g4))]
    return expr - sum(terms_higher)
def remove_cons(expr):
    terms_higher = [
        term for term in expr.args if (term.has(a) or term.has(ad)) ]
    
    return sum(terms_higher)

In [12]:
def replace_scalars_with_valued_matrices(expr, matrix_values):
        """
        Replace scalar symbols and scalar operations in a SymPy scalar expression with valued matrices.

        Args:
        expr (sympy.Expr): The input scalar expression.
        matrix_values (dict): A dictionary mapping scalar symbols to valued matrices.

        Returns:
        sympy.Expr: The modified expression with valued matrices.
        """

        def map_operations(node):
            if (isinstance(node, AnnihilateBoson) or isinstance(node, CreateBoson) )and node in matrix_values:
                return matrix_values[node]
            elif isinstance(node, sp.Add):
                return sp.Add(*[map_operations(arg) for arg in node.args])
            elif isinstance(node, sp.Mul):
                return sp.Mul(*[map_operations(arg) for arg in node.args])
            elif isinstance(node, sp.Pow):
                base, exponent = map_operations(node.base), map_operations(node.exp)
                return base**exponent
            else:
                return node

        return map_operations(expr)

In [13]:
# Initialize the annihilation (lowering) and creation (raising) matrices with zeros
from sympy import symbols, sqrt
n=15
s = sp.zeros(n)
s_dagger = sp.zeros(n)

# Fill the matrices using sympy functions
for i in range(n-1):
    s[i, i+1] = sqrt(i+1)
    s_dagger[i+1, i] = sqrt(i+1)

# Extract the upper triangular part of the matrix
# upper_tri = g_matrix.upper_triangular()

# # Extract the lower triangular part of the matrix
# lower_tri = g_matrix.lower_triangular()


In [15]:
matrix_values = {a: s, ad: s_dagger}
g_matrix = replace_scalars_with_valued_matrices(remove_cons((a+ad)).expand(), matrix_values)

In [17]:
g_matrix

Matrix([
[0,       1,       0,       0,       0,       0,       0,         0,         0,        0,        0,         0,         0,        0,        0],
[1,       0, sqrt(2),       0,       0,       0,       0,         0,         0,        0,        0,         0,         0,        0,        0],
[0, sqrt(2),       0, sqrt(3),       0,       0,       0,         0,         0,        0,        0,         0,         0,        0,        0],
[0,       0, sqrt(3),       0,       2,       0,       0,         0,         0,        0,        0,         0,         0,        0,        0],
[0,       0,       0,       2,       0, sqrt(5),       0,         0,         0,        0,        0,         0,         0,        0,        0],
[0,       0,       0,       0, sqrt(5),       0, sqrt(6),         0,         0,        0,        0,         0,         0,        0,        0],
[0,       0,       0,       0,       0, sqrt(6),       0,   sqrt(7),         0,        0,        0,         0,         0,        0,  

In [18]:
from sympy import symbols, Matrix
# Define the symbols
omega, alpha, omega_1, omega_2,omegap = symbols('omega alpha omega_1 omega_2 omega_p', real=True)

# Define the epsilon_i values
epsilon = [i * omegap + i*(i-1) * alpha / 2 for i in range(n)]

# Initialize the delta_1 matrix with zeros
delta_1 = Matrix.zeros(n, n)

# Fill the matrix according to the given formula and specific epsilon_i values
for i in range(n):
    for j in range(n):
        if i != j:
            delta_1[i, j] = 1 / (epsilon[i] - epsilon[j] - omega_1)
            
delta_2 = Matrix.zeros(n, n)

# Fill the matrix according to the given formula and specific epsilon_i values for omega_2
for i in range(n):
    for j in range(n):
        if i != j:
            delta_2[i, j] = 1 / (epsilon[i] - epsilon[j] - omega_2)


In [19]:
chim1 = Matrix.zeros(n, n)
chim2 = Matrix.zeros(n, n)
for i in range(n):
    for j in range(n):
        if delta_1[i, j] != 0:
            element = g_matrix[i, j]
            if sp.im(element) != 0 and sp.re(element) == 0:  # Pure imaginary
                magnitude = sp.im(element)
                chim1[i, j] = (magnitude**2 * delta_1[i, j])
                chim2[i, j] = (magnitude**2 * delta_2[i, j])
            else:  # Real or complex
                chim1[i, j] = (element**2 * delta_1[i, j])
                chim2[i, j] = (element**2 * delta_2[i, j])
        else:
            chim1[i, j] = 0
            chim2[i, j] = 0

In [20]:
chi0 = sum(chim1[0, j] for j in range(n))-sum(chim1[i, 0] for i in range(n))

In [21]:
chi0 = remove_higher((chi0.expand()))

In [22]:
chi0

-1/(-omega_1 + omega_p) + 1/(-omega_1 - omega_p)

In [23]:
disp = sum(chim1[1, j] for j in range(0,n))-sum(chim1[i, 1] for i in range(0,n))- (sum(chim1[0, j] for j in range(n))-sum(chim1[i, 0] for i in range(n)))

In [24]:
disp = remove_higher((disp.expand()))

In [30]:
disp.simplify().factor()

-4*alpha*(alpha*omega_p + omega_1**2 + omega_p**2)/((omega_1 - omega_p)*(omega_1 + omega_p)*(alpha - omega_1 + omega_p)*(alpha + omega_1 + omega_p))

In [29]:
disp.simplify()

-2/(alpha + omega_1 + omega_p) - 2/(alpha - omega_1 + omega_p) + 2/(omega_1 + omega_p) - 2/(omega_1 - omega_p)

In [ ]:
correction = 0

In [120]:
disp = (disp.subs(omegap,correction+omega)).subs(alpha,correction)

In [121]:
E =disp
x = symbols('x', real=True)
y= symbols('y', real=True)
E=E.subs(g3**2,x*omega**2)
E=E.subs(g4,y*omega)
from sympy import symbols, diff
f_0 = E.subs({x: 0, y: 0})
df_dx = diff(E, x).subs({x: 0, y: 0}) * (x - 0)
df_dy = diff(E, y).subs({x: 0, y: 0}) * (y - 0)

taylor_first_order = f_0 + df_dx + df_dy

disp = taylor_first_order.expand().subs(y,g4/omega).subs(x,g3**2/omega**2)

In [122]:
temp = 0
for term in disp.args:
    temp += term.simplify()
temp

-120*g3**2/(omega*(omega**2 + 2*omega*omega_1 + omega_1**2)) - 120*g3**2/(omega*(omega**2 - 2*omega*omega_1 + omega_1**2)) - 64*g3**2/(omega**2*(2*omega + omega_1)) - 64*g3**2/(omega**2*(2*omega - omega_1)) + 152*g3**2/(omega**2*(omega + omega_1)) + 152*g3**2/(omega**2*(omega - omega_1)) + 24*g4/(omega**2 + 2*omega*omega_1 + omega_1**2) + 24*g4/(omega**2 - 2*omega*omega_1 + omega_1**2) - 24*g4/(omega*(omega + omega_1)) - 24*g4/(omega*(omega - omega_1))

In [123]:
ind_b = symbols('c')
b = B(ind_b)
bd = Bd(ind_b)
omega_b = symbols('omega_1', real=True)
def _m_normal(expr):
    if not expr.is_Mul:
        return expr
    else:
        c_part = []
        string = []
        for factor in expr.args:
            if factor.is_commutative:
                c_part.append(factor)
            else:
                string.append(factor)
                
        expr_a = 1
        expr_b = 1
        expr_c = 1
        
        for factor in string:
            if factor.is_Pow:
                if factor.args[0].args[0] == ind_a:
                    expr_a *= factor
                elif factor.args[0].args[0] == ind_b:
                    expr_b *= factor
                else:
                    expr_c *= factor
            else:
                if factor.args[0] == ind_a:
                    expr_a *= factor
                elif factor.args[0] == ind_b:
                    expr_b *= factor
                else:
                    expr_c *= factor
                    
        if expr_a != 1:
            expr_a = normal(expr_a)
        if expr_b != 1:
            expr_b = normal(expr_b)
        if expr_c != 1:
            expr_c = normal(expr_c)
    
        return sp.Mul(*c_part) * expr_a * expr_b * expr_c    

def m_normal(expr):
    expr = expr.expand()
    if isinstance(expr, sp.Add):
        return sp.Add(*[_m_normal(term) for term in expr.args])
    else:
        return _m_normal(expr)

In [210]:
from sympy.physics.quantum import TensorProduct
nb=5
b = sp.zeros(nb)
b_dagger = sp.zeros(nb)
for i in range(nb-1):
    b[i, i+1] = sqrt(i+1)
    b_dagger[i+1, i] = sqrt(i+1)
V = TensorProduct(Matrix(g_matrix),Matrix( b+b_dagger))
# V = TensorProduct(Matrix(upper_tri),Matrix( b_dagger))+TensorProduct(Matrix(lower_tri),Matrix( b))
ENN = [
    i * omegap + i*(i-1) * alpha / 2 + j * omega_b
    for i in range(n)
    for j in range(nb)
]

In [211]:

import sympy as sp


# Matrix elements of V in the eigenbasis of H0
V_ij = V

# Compute the fourth order energy correction for the ground state |0>

def energy_sec(N):
    E2_N = 0  # Initialize the fourth order correction
    EN = ENN[N]
    dim = n*nb
    for m in range(dim):
        if m != N:
            VNm = V_ij[m, N]
            if sp.im(VNm) != 0 and sp.re(VNm) == 0:  # Pure imaginary
                VNm = sp.im(VNm)
            E2_N = E2_N + (VNm**2/(EN - ENN[m])) 
    return E2_N

def energy_four(N):
    E4_N = 0  # Initialize the fourth order correction
    EN = ENN[N]
    dim = n*nb
    for m in range(dim):
        if m != N and V_ij[N, m] !=0 :
            for p in range(dim):
                if p != N and V_ij[m, p]!=0 :
                    for l in range(dim):
                        if l != N and V_ij[p, l]!=0 and V_ij[l, N]!=0:
                            # Contribution from the main term
                            E4_N += ((V_ij[N, m] * V_ij[m, p] * V_ij[p, l] * V_ij[l, N] /
                                     ((EN - ENN[m]) * (EN - ENN[p]) * (EN - ENN[l]))))

    for m in range(dim):
        if m != N:
            for p in range(dim):
                if p != N:
                    VNm = V_ij[N, m]
                    VNp = V_ij[N, p]
                    if VNm == 0 or VNp ==0:
                        continue
                    if sp.im(VNm) != 0 and sp.re(VNm) == 0:  # Pure imaginary
                        VNm = sp.im(VNm)
                    if sp.im(VNp) != 0 and sp.re(VNp) == 0:  # Pure imaginary    
                        VNp = sp.im(VNp)
                    E4_N = E4_N - ((VNm**2/(EN - ENN[m])**2) * (VNp**2/(EN - ENN[p])))
    return E4_N


In [212]:
ec = energy_sec(1) - energy_sec(0)
ec = (ec.subs(omegap,correction+omega)).subs(alpha,correction)
E =ec
x = symbols('x', real=True)
y= symbols('y', real=True)
E=E.subs(g3**2,x*omega**2)
E=E.subs(g4,y*omega)
from sympy import symbols, diff
f_0 = E.subs({x: 0, y: 0})
df_dx = diff(E, x).subs({x: 0, y: 0}) * (x - 0)
df_dy = diff(E, y).subs({x: 0, y: 0}) * (y - 0)

taylor_first_order = f_0 + df_dx + df_dy

ec = taylor_first_order.expand().subs(y,g4/omega).subs(x,g3**2/omega**2)

In [213]:
temp = 0
for term in ec.args:
    temp += term.simplify()
temp

-60*g3**2/(omega*(omega**2 + 2*omega*omega_1 + omega_1**2)) - 60*g3**2/(omega*(omega**2 - 2*omega*omega_1 + omega_1**2)) - 32*g3**2/(omega**2*(2*omega + omega_1)) - 32*g3**2/(omega**2*(2*omega - omega_1)) + 76*g3**2/(omega**2*(omega + omega_1)) + 76*g3**2/(omega**2*(omega - omega_1)) + 12*g4/(omega**2 + 2*omega*omega_1 + omega_1**2) + 12*g4/(omega**2 - 2*omega*omega_1 + omega_1**2) - 12*g4/(omega*(omega + omega_1)) - 12*g4/(omega*(omega - omega_1)) - 1/(omega + omega_1) - 1/(omega - omega_1)

In [214]:
from qutip import qsave
qsave([temp],'/home/yunwei/SNAIL/charge-coupling/Ben/ec')

In [215]:
disp = energy_sec(nb+1) - energy_sec(nb) - energy_sec(1)+energy_sec(0)

In [217]:
disp  = remove_higher((disp.expand()))

In [218]:
disp = (disp.subs(omegap,correction+omega)).subs(alpha,correction)

In [219]:
E =disp
x = symbols('x', real=True)
y= symbols('y', real=True)
E=E.subs(g3**2,x*omega**2)
E=E.subs(g4,y*omega)
from sympy import symbols, diff
f_0 = E.subs({x: 0, y: 0})
df_dx = diff(E, x).subs({x: 0, y: 0}) * (x - 0)
df_dy = diff(E, y).subs({x: 0, y: 0}) * (y - 0)

taylor_first_order = f_0 + df_dx + df_dy

disp = taylor_first_order.expand().subs(y,g4/omega).subs(x,g3**2/omega**2)

In [220]:
temp = 0
for term in disp.args:
    temp += term.simplify()
temp

-120*g3**2/(omega*(omega**2 + 2*omega*omega_1 + omega_1**2)) - 120*g3**2/(omega*(omega**2 - 2*omega*omega_1 + omega_1**2)) - 64*g3**2/(omega**2*(2*omega + omega_1)) - 64*g3**2/(omega**2*(2*omega - omega_1)) + 152*g3**2/(omega**2*(omega + omega_1)) + 152*g3**2/(omega**2*(omega - omega_1)) + 24*g4/(omega**2 + 2*omega*omega_1 + omega_1**2) + 24*g4/(omega**2 - 2*omega*omega_1 + omega_1**2) - 24*g4/(omega*(omega + omega_1)) - 24*g4/(omega*(omega - omega_1))

In [221]:
from qutip import qsave
qsave([temp],'/home/yunwei/SNAIL/charge-coupling/Ben/ds')

In [222]:
kerr = energy_four(2) - 2*energy_four(1) + energy_four(0)

In [223]:
kerr = (kerr.subs(omegap,correction+omega)).subs(alpha,correction)

In [224]:
kerr = remove_higher(kerr)

In [225]:
terms = kerr.as_ordered_terms()

# Process each term
new_terms = []
for term in terms:
    # Check if the term is a rational expression
    if term.is_rational_function():
        # Extract the numerator and the denominator
        num, denom = term.as_numer_denom()
        # Expand the numerator
        expanded_num = sp.expand(num)
        # Recombine with the original denominator
        new_term = expanded_num / denom
        new_terms.append(new_term)
    else:
        # If not a rational function, just keep the term as is
        new_terms.append(term)

# Sum the processed terms to form the new 'kerr' expression
new_kerr = sum(new_terms)


In [226]:
kerr = new_kerr

In [227]:
E =kerr
x = symbols('x', real=True)
y= symbols('y', real=True)
E=E.subs(g3**2,x*omega**2)
E=E.subs(g4,y*omega)
from sympy import symbols, diff
f_0 = E.subs({x: 0, y: 0})
df_dx = diff(E, x).subs({x: 0, y: 0}) * (x - 0)
df_dy = diff(E, y).subs({x: 0, y: 0}) * (y - 0)

taylor_first_order = 0 + df_dx + df_dy

kerr = taylor_first_order.expand().subs(y,g4/omega).subs(x,g3**2/omega**2)

In [228]:
terms = kerr.as_ordered_terms()

# Process each term
new_terms = []
for term in terms:
    # Check if the term is a rational expression
    if term.is_rational_function():
        # Extract the numerator and the denominator
        num, denom = term.as_numer_denom()
        # Simplify the denominator
        simplified_denom = sp.factor(denom)
        # Recombine with the original numerator
        new_term = num / simplified_denom
        new_terms.append(new_term)
    else:
        # If not a rational function, just keep the term as is
        new_terms.append(term)

# Sum the processed terms to form the new 'kerr' expression
new_kerr = sum(new_terms)

In [229]:
new_kerr

-60*g3**2/(omega*(omega + omega_1)**4) + 240*g3**2/(omega*(omega - omega_1)*(omega + omega_1)**3) + 240*g3**2/(omega*(omega - omega_1)**2*(omega + omega_1)**2) + 240*g3**2/(omega*(omega - omega_1)**3*(omega + omega_1)) - 60*g3**2/(omega*(omega - omega_1)**4) - 120*g3**2/(omega*omega_1*(omega + omega_1)**3) + 120*g3**2/(omega*omega_1*(omega - omega_1)**3) - 192*g3**2/(omega**2*(2*omega + omega_1)**2*(3*omega + 2*omega_1)) - 192*g3**2/(omega**2*(2*omega - omega_1)**2*(3*omega - 2*omega_1)) - 128*g3**2/(omega**2*(omega + 2*omega_1)*(2*omega + omega_1)**2) - 384*g3**2/(omega**2*(omega + omega_1)*(2*omega + omega_1)*(3*omega + 2*omega_1)) + 64*g3**2/(omega**2*(omega + omega_1)*(2*omega + omega_1)**2) + 64*g3**2/(omega**2*(omega + omega_1)*(2*omega - omega_1)**2) - 192*g3**2/(omega**2*(omega + omega_1)**2*(3*omega + 2*omega_1)) + 108*g3**2/(omega**2*(omega + omega_1)**2*(3*omega + omega_1)) + 64*g3**2/(omega**2*(omega + omega_1)**2*(2*omega + omega_1)) + 64*g3**2/(omega**2*(omega + omega_1)*

In [230]:
from qutip import qsave
qsave([new_kerr],'/home/yunwei/SNAIL/charge-coupling/Ben/sk')

In [231]:
from sympy.physics.quantum import TensorProduct
from sympy import eye
nb1=4
nb2=4
omega_b1 = symbols('omega_c1', real=True)
omega_b2 = symbols('omega_c2', real=True)
b = sp.zeros(nb1)
b_dagger = sp.zeros(nb1)
for i in range(nb1-1):
    b[i, i+1] = sqrt(i+1)
    b_dagger[i+1, i] = sqrt(i+1)
    
# v1 = TensorProduct(Matrix(upper_tri),Matrix( b_dagger))+TensorProduct(Matrix(lower_tri),Matrix(b))
# v1 = TensorProduct(v1,eye(nb1))
# v2up = TensorProduct(Matrix(upper_tri),eye(nb1))
# v2lo = TensorProduct(Matrix(lower_tri),eye(nb1))
# v2 = TensorProduct(v2up,Matrix( b_dagger))+TensorProduct(v2lo,Matrix( b))


v1 = TensorProduct(Matrix(g_matrix),Matrix( b+b_dagger))
v1 = TensorProduct(v1,eye(nb1))
v2 = TensorProduct(Matrix(g_matrix),eye(nb1))
v2 = TensorProduct(v2,Matrix( b+b_dagger))
V=v1+v2
ENN = [
    i * omegap - i*(i-1) * alpha / 2 + j * omega_b1+l * omega_b2
    for i in range(n)
    for j in range(nb1)
    for l in range(nb2)
]

In [232]:
def energy_four(N):
    V_ij = V
    E4_N = 0  # Initialize the fourth order correction
    EN = ENN[N]
    dim = n*nb1*nb2
    for m in range(dim):
        if m != N and V_ij[N, m] !=0 :
            for p in range(dim):
                if p != N and V_ij[m, p]!=0 :
                    for l in range(dim):
                        if l != N and V_ij[p, l]!=0 and V_ij[l, N]!=0:
                            # Contribution from the main term
                            E4_N += (((V_ij[N, m] * V_ij[m, p] * V_ij[p, l] * V_ij[l, N] /
                                     ((EN - ENN[m]) * (EN - ENN[p]) * (EN - ENN[l]))))).simplify()

    for m in range(dim):
        if m != N:
            for p in range(dim):
                if p != N:
                    VNm = V_ij[N, m]
                    VNp = V_ij[N, p]
                    if VNm == 0 or VNp ==0:
                        continue
                    if sp.im(VNm) != 0 and sp.re(VNm) == 0:  # Pure imaginary
                        VNm = sp.im(VNm)
                    if sp.im(VNp) != 0 and sp.re(VNp) == 0:  # Pure imaginary    
                        VNp = sp.im(VNp)
                    E4_N = E4_N - ((VNm**2/(EN - ENN[m])**2) * (VNp**2/(EN - ENN[p]))).simplify()
    return E4_N

In [233]:
ck = energy_four(nb1+1) - energy_four(nb1) - energy_four(1) + energy_four(0)

In [234]:
ck = (ck.subs(omegap,correction+omega)).subs(alpha,correction)

In [235]:
terms = ck.as_ordered_terms()

# Process each term
new_terms = []
for term in terms:
    # Check if the term is a rational expression
    if term.is_rational_function():
        # Extract the numerator and the denominator
        num, denom = term.as_numer_denom()
        # Simplify the denominator
        simplified_denom = sp.factor(denom)
        # Recombine with the original numerator
        new_term = num / simplified_denom
        new_terms.append(new_term)
    else:
        # If not a rational function, just keep the term as is
        new_terms.append(term)

# Sum the processed terms to form the new 'kerr' expression
ck = sum(new_terms)

In [236]:
E =ck
x = symbols('x', real=True)
y= symbols('y', real=True)
E=E.subs(g3**2,x*omega**2)
E=E.subs(g4,y*omega)
from sympy import symbols, diff
f_0 = E.subs({x: 0, y: 0})
df_dx = diff(E, x).subs({x: 0, y: 0}) * (x - 0)
df_dy = diff(E, y).subs({x: 0, y: 0}) * (y - 0)

taylor_first_order = 0 + df_dx + df_dy

ck = taylor_first_order.expand().subs(y,g4/omega).subs(x,g3**2/omega**2)

In [237]:
terms = ck.as_ordered_terms()

# Process each term
new_terms = []
for term in terms:
    # Check if the term is a rational expression
    if term.is_rational_function():
        # Extract the numerator and the denominator
        num, denom = term.as_numer_denom()
        # Simplify the denominator
        simplified_denom = sp.factor(denom)
        # Recombine with the original numerator
        new_term = num / simplified_denom
        new_terms.append(new_term)
    else:
        # If not a rational function, just keep the term as is
        new_terms.append(term)

# Sum the processed terms to form the new 'kerr' expression
new_ck = sum(new_terms)

In [238]:
new_ck

-120*g3**2/(omega*(omega + omega_c2)**2*(2*omega + omega_c1 + omega_c2)**2) - 120*g3**2/(omega*(omega + omega_c2)**2*(2*omega - omega_c1 + omega_c2)**2) - 240*g3**2/(omega*(omega + omega_c2)**3*(2*omega + omega_c1 + omega_c2)) - 240*g3**2/(omega*(omega + omega_c2)**3*(2*omega - omega_c1 + omega_c2)) - 120*g3**2/(omega*(omega + omega_c2)**3*(omega_c1 + omega_c2)) + 120*g3**2/(omega*(omega + omega_c2)**3*(omega_c1 - omega_c2)) - 120*g3**2/(omega*(omega - omega_c2)**2*(2*omega + omega_c1 - omega_c2)**2) - 120*g3**2/(omega*(omega - omega_c2)**2*(2*omega - omega_c1 - omega_c2)**2) - 240*g3**2/(omega*(omega - omega_c2)**3*(2*omega + omega_c1 - omega_c2)) - 240*g3**2/(omega*(omega - omega_c2)**3*(2*omega - omega_c1 - omega_c2)) + 120*g3**2/(omega*(omega - omega_c2)**3*(omega_c1 + omega_c2)) - 120*g3**2/(omega*(omega - omega_c2)**3*(omega_c1 - omega_c2)) - 240*g3**2/(omega*(omega + omega_c1)*(omega + omega_c2)*(2*omega + omega_c1 + omega_c2)**2) - 240*g3**2/(omega*(omega + omega_c1)*(omega + o

In [239]:
from qutip import qsave
qsave([new_ck],'/home/yunwei/SNAIL/charge-coupling/Ben/ck')